In [1]:
# define the set of commands we want to classify with hmm
import librosa
import numpy as np 
import os

commands = ["demo_fider_ac", # 0
            "demo_fider_bilgileri", # 1 
            "demo_fider_kapat", # 2
            "nem_durumu", # 3
            "gsm_durumu" # 4
            ]
# since we have multiple .wav files in the dirs
data = []
labels = []

dataset_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands"
for i, command in enumerate(commands):
    # loop through each file in the folder
    for file in os.listdir(dataset_path):
        if file.endswith(".wav"):
            file_path = os.path.join(dataset_path, file) # there's need to have the full path to the file.
            # print(file_path)
            audio_data, sr = librosa.load(file_path, sr=None)

            # print("File:", filename)
            # print("Sample rate:", sr)
            # print("Number of samples:", len(audio_data))

            mfcc = librosa.feature.mfcc(
                y=audio_data,
                sr=sr,
                n_mfcc=13
            )

            data.append(mfcc.T)
            labels.append(i)
        
# save the features and labels to a .npy file
# save an array to a binary file
np.save("data.npy", np.vstack(data))
np.save("labels.npy", np.array(labels))

In [2]:
# extract the features from each file: mfcc features & save them along with their labels
data = np.load("data.npy", allow_pickle=True)
data_reshaped = data.ravel()[:100]
labels = np.load("labels.npy", allow_pickle=True)
print(len(labels), len(data), len(data_reshaped))

# param for mfcc extraction
n_mfcc = 15

# preprocess the .wav file: extract mfcc features and save them with their labe
features = []
for i in range(len(data_reshaped)): # len(data) = 24615
    mfcc = librosa.feature.mfcc(
        y = data[i],
        sr = 44100,
        n_mfcc= n_mfcc
    )

    features.append((mfcc.T, labels[i])) 
    # we run into a size error: IndexError: index 100 is out of bounds for axis 0 with size 100
    # labels = 100, 24615 > 100 (actually 99: 0 to 99 = 100): soution reshape data to match 100:
    # may reduce the accuracy of our model

100 24615 100


/home/ix502iv/miniconda3/envs/TensorFlow/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=13
  return f(*args, **kwargs)
